In [ ]:
%matplotlib inline

Attribution: 

   * Most material is adapted from [Morvan Zhou's Tutorials](https://morvanzhou.github.io/tutorials/)

# AutoEncoders

We are going to apply what we've learned about supervised learning in PyTorch to construct our first unsupervised learning model, an AutoEncoder.

In [ ]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.utils.data as Data
import torchvision
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
import numpy as np

In [ ]:
torch.manual_seed(1)    # make reproducible

In [ ]:
# Hyper Parameters
num_epochs = 10
batch_size = 64
learning_rate = 0.005         # learning rate
n_test_img = 5

In [ ]:
# Mnist digits dataset
train_data = torchvision.datasets.MNIST(
    root='./data/',
    train=True,                                     # this is training data
    transform=torchvision.transforms.ToTensor(),    # Converts a PIL.Image or numpy.ndarray to
                                                    # torch.FloatTensor of shape (C x H x W) and normalizes in the range [0.0, 1.0]
    download=True,                                  # download it if you don't have it
)

The MNIST dataset consists of grayscale, 28x28 images of handwritten digits.

The classic split is 60,000 training examples and 10,000 test examples.

Let's have a look at some of the MNIST data:

In [ ]:
train_data.train_data[0:100].view(100,1,28,28).size()

print(train_data.train_data.size())     # (60000, 28, 28)
print(train_data.train_labels.size())   # (60000)

n_toplot = 64

plt.imshow(torchvision.utils.make_grid(train_data.train_data[0:n_toplot].view(n_toplot,1,28,28), 
                                       nrow=int(np.sqrt(n_toplot))).numpy().transpose(1,2,0))

Let's set up a Data Loader to return batches of size `batch_size`. The size of each batch will be (`batch_size`, 1, 28, 28):

In [ ]:
train_loader = Data.DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)

We are now going to define a classical "bottleneck"-style deep autoencoder which gradually reduces the dimensionality of the input until it is compressed to a 3-dimensional code, then gradually expands back to reconstruct:

In [ ]:
class AutoEncoder(nn.Module):
    def __init__(self):
        super(AutoEncoder, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 128),
            nn.Tanh(),
            nn.Linear(128, 64),
            nn.Tanh(),
            nn.Linear(64, 12),
            nn.Tanh(),
            nn.Linear(12, 3),   # compress to 3 features which can be visualized in plt
        )
        self.decoder = nn.Sequential(
            nn.Linear(3, 12),
            nn.Tanh(),
            nn.Linear(12, 64),
            nn.Tanh(),
            nn.Linear(64, 128),
            nn.Tanh(),
            nn.Linear(128, 28 * 28),
            nn.Sigmoid(),       # compress to a range (0, 1)
        )

    def forward(self, x):
        encoding = self.encoder(x)
        decoding = self.decoder(encoding)
        return encoding, decoding

In [ ]:
autoencoder = AutoEncoder()
print(autoencoder)

optimizer = torch.optim.Adam(autoencoder.parameters(), lr=learning_rate)
loss_func = nn.MSELoss()

# original data (first row) for viewing
view_data = Variable(train_data.train_data[:n_test_img].view(-1, 28*28).type(torch.FloatTensor)/255.)

for epoch in range(num_epochs):
    for step, (x, y) in enumerate(train_loader):
        b_x = Variable(x.view(-1, 28*28))   # batch x, shape (batch, 28*28)
        b_y = Variable(x.view(-1, 28*28))   # batch y, shape (batch, 28*28)
        b_label = Variable(y)               # batch label

        encoded, decoded = autoencoder(b_x)

        loss = loss_func(decoded, b_y)      # mean square error
        optimizer.zero_grad()               # clear gradients for this training step
        loss.backward()                     # backpropagation, compute gradients
        optimizer.step()                    # apply gradients

        if step % 500 == 0 and epoch in [0, 5, num_epochs-1]:
            print('epoch: ', epoch, '| train loss: %.4f' % loss.data[0])

            # plotting decoded image (second row)
            _, decoded_data = autoencoder(view_data)
            
            # initialize figure
            f, a = plt.subplots(2, n_test_img, figsize=(5, 2))
            
            for i in range(n_test_img):
                a[0][i].imshow(np.reshape(view_data.data.numpy()[i], (28, 28)), cmap='gray'); 
                a[0][i].set_xticks(()) 
                a[0][i].set_yticks(())
    
            for i in range(n_test_img):
                a[1][i].clear()
                a[1][i].imshow(np.reshape(decoded_data.data.numpy()[i], (28, 28)), cmap='gray')
                a[1][i].set_xticks(()) 
                a[1][i].set_yticks(())
            plt.show(); plt.pause(0.05)
            

Since the bottleneck layer is 3-d, we can visualize the data (here, the first 200 points) by projecting it to this space.

In [ ]:
n_vis = 200
view_data = Variable(train_data.train_data[:n_vis].view(-1, 28*28).type(torch.FloatTensor)/255.)
encoded_data, _ = autoencoder(view_data)
fig = plt.figure(2); ax = Axes3D(fig)
X, Y, Z = encoded_data.data[:, 0].numpy(), encoded_data.data[:, 1].numpy(), encoded_data.data[:, 2].numpy()
values = train_data.train_labels[:n_vis].numpy()
for x, y, z, s in zip(X, Y, Z, values):
    c = cm.rainbow(int(255*s/9)); ax.text(x, y, z, s, backgroundcolor=c)
ax.set_xlim(X.min(), X.max()); ax.set_ylim(Y.min(), Y.max()); ax.set_zlim(Z.min(), Z.max())
plt.show()

#### Exercises

1. Noting that the outputs are sigmoid, what happens when you use a binary cross entropy loss instead of MSE?

2. Try changing from tanh to ReLU hidden units. Does this make a difference?

3. Implement a denoising autoencoder.

4. Implement a contractive autoencoder. The tricky part here is implementing the loss term based on the Frobenius norm of the Jacobian. [Agustinus Kristiadi's Blog](https://wiseodd.github.io/techblog/2016/12/05/contractive-autoencoder/) gives a nice derivation of this term and what it looks like in code.